In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

### BERT-based-uncased

In [20]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# obtain BERT encoding

cnt = 0
path = 'dataset_embeddings/bertagnewstest.txt'      # path
ds = textattack.datasets.HuggingFaceDataset('ag_news', split='test')                               # train dataset
#path = 'dataset_embeddings/bertmrtest.txt'      # path
#ds = test_dataset                               # train dataset

st = datetime.datetime.now()

with open(path, 'w') as f:
    for text, label in ds:
        cnt += 1
        t = bert_tokenizer(text['text'], return_tensors="pt", padding=True, truncation=True)
        t = bert_model(**t).pooler_output.detach().numpy()
        
        np.savetxt(f, np.append(t, label).reshape(1,-1), delimiter=',')
        f.write('\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

textattack: Loading datasets dataset ag_news, split test.


.......0:13:04.945165


### DistilBERT

In [73]:
# Distilbert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
# obtain DistilBERT encoding

cnt = 0
path = 'dataset_embeddings/disbertagnewstrain.txt'      # path
ds = textattack.datasets.HuggingFaceDataset('ag_news', split='train')                               # train dataset
#path = 'dataset_embeddings/bertmrtest.txt'      # path
#ds = test_dataset                               # train dataset

st = datetime.datetime.now()

with open(path, 'w') as f:
    for text, label in ds:
        cnt += 1
        t = disbert_tokenizer(text['text'], return_tensors="pt", padding=True, truncation=True)
        t = disbert_model(**t).last_hidden_state.detach().numpy().mean(axis=1)
        
        np.savetxt(f, np.append(t, label).reshape(1,-1), delimiter=',')
        f.write('\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

textattack: Loading datasets dataset ag_news, split train.


........................................................................................................................1:47:00.250764


### CLIP

In [7]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [81]:
# obtain clip encoding

cnt = 0
path = 'dataset_embeddings/clipagnewstest.txt'      # path
ds = textattack.datasets.HuggingFaceDataset('ag_news', split='test')                               # train dataset

st = datetime.datetime.now()

with open(path, 'w') as f:
    for text, label in ds:
        cnt += 1
        t = clip_processor(text['text'], return_tensors="pt", padding=True, truncation=True)
        t = text_encoder(**t).pooler_output.detach().numpy()
        
        np.savetxt(f, np.append(t, label).reshape(1,-1), delimiter=',')
        f.write('\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

textattack: Loading datasets dataset ag_news, split test.


.......0:08:27.974111
